In [1]:
from google.cloud import storage
import tensorflow as tf
from PIL import Image
import numpy as np


BUCKET_NAME = "ml-models-cxrlton"
class_names = ["Early Blight", "Late Blight","Healthy"]

model = None

def download_blob(bucket_name,sourcre_blob_name,destination_file_name):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)
    
    
    
def predict(request):
    global model
    if model is not None:
        download_blob(
            BUCKET_NAME,
            "models/ImageGen.h5",
            "/tmp/ImageGen.h5",
    )
        model = tf.keras.load_model('/tmp/ImageGen.h5')
    image = request.files["file"]
    image = np.array(
        Image.open(image).convert("RGB").resize((256, 256)) # image resizing
    )
    image = image/255 # normalize the image in 0 to 1 range

    img_array = tf.expand_dims(img, 0)
    predictions = model.predict(img_array)

    print("Predictions:",predictions)

    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)

    return {"class": predicted_class, "confidence": confidence}

2023-10-21 15:22:25.785868: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
